# Three Models for location

Imports

In [ ]:
# Import packages for data preparation
import numpy as np
import tensorflow as tf
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

# Import packages for modeling
import tensorflow as tf
import tensorflow_hub as hub
import datetime
import csv
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import datasets, layers, models, optimizers
# Load the TensorBoard notebook extension
%load_ext tensorboard

## Loading data

In [ ]:
#Path to images
image_dir = "../images/"

#Load train-data
train_data = pd.read_csv('../data/train_corrected.csv')
train_data.image_id = train_data.image_id.apply(lambda x: x.strip()+".JPG")
train_data.image_location = train_data.image_location.apply(lambda x: x.lower())

#Load test data
test_data = pd.read_csv('../data/test_corrected.csv')
test_data.image_id = test_data.image_id.apply(lambda x: x.strip()+".JPG")
test_data.image_location = test_data.image_location.apply(lambda x: x.lower())

#Split train data using image location
train_data_left = train_data.loc[train_data['image_location']=='left']
train_data_right = train_data.loc[train_data['image_location']=='right']
train_data_top = train_data.loc[train_data['image_location']=='top']

#Get unique_turtle_ids from train.csv
unique_turtle_ids = list(train_data['turtle_id'].unique())
#Add category for new turtle for test set
unique_turtle_ids.append("new_turtle")

#Get number of images for train/test split
split = 0.9
lines = round(min(len(train_data_left),len(train_data_right),len(train_data_top))*split)

#We set some parameters for the model
HEIGHT = 224 #image height
WIDTH = 224 #image width
CHANNELS = 3 #image RGB channels
CLASS_NAMES = unique_turtle_ids
NCLASSES = len(CLASS_NAMES)
BATCH_SIZE = 32
SHUFFLE_BUFFER = 10 * BATCH_SIZE
TRAINING_SIZE = lines
TRAINING_STEPS = (TRAINING_SIZE // BATCH_SIZE)
AUTOTUNE = tf.data.experimental.AUTOTUNE

## Preparing data for model

In [ ]:
def preprocess(augment = True):
    '''
    Function to create ImageDataGenerator-Object to augment and scale image
    input: augment=True
    output: train_datagen, test_datagen
    If augment is true, augmentation is applied on train_datagen, scaling for test_datagen.
    If augment is false, only scaling is applied for both generators.
    '''
    if augment == True:
        train_datagen = ImageDataGenerator(
                rotation_range     = 40,
                width_shift_range  = 0.2,
                height_shift_range = 0.2,
                # use "rescale" to scale array of original image pixel values to be between [0,1] and specify the parameter rescale=1./255.
                rescale            = 1./255, 
                shear_range        = 0.2,
                zoom_range         = 0.2,
                horizontal_flip    = False,
                fill_mode          = 'nearest')

        test_datagen = ImageDataGenerator(rescale=1./255)
    
    else:
        train_datagen = ImageDataGenerator(rescale=1./255)
        test_datagen  = ImageDataGenerator(rescale=1./255)
        
    return train_datagen, test_datagen


def use_image_generator(df, train_datagen, test_datagen, training=True): 
    '''
    Function to apply ImageDataGenerator-Object to images for augmentation and scaling
    input: 
        dataframe for which the function should be used
        train_datagen as ImageDataGenerator-object
        test_datagen as ImageDataGenerator-object
        training=True
    output: train_generator and validation_generator or test_generator
    If training is true, train_generator (augmented image and label) and validation_generator (scaled image and label) are returned.
    If training is false, test_generator is returned containing scaled image, no label is returned.
    '''
    if training == True:
        # Augment and scale images for training
        # This is a generator that will read pictures found in directory, 
        # and indefinitely generate batches of augmented image data
        # flow_from_directory: Takes the path to a directory & generates batches of augmented data.
        train_generator = train_datagen.flow_from_dataframe(dataframe = df[0:lines], 
                directory   = image_dir,
                x_col       = "image_id", #name of column(in dataframe) having file names
                y_col       = "turtle_id", #name of column(in dataframe) having class/label
                target_size = (HEIGHT, WIDTH),
                batch_size  = BATCH_SIZE,
                classes     = CLASS_NAMES,
                class_mode  = 'categorical',
                shuffle     = True)

        # Scale images for validation
        validation_generator = test_datagen.flow_from_dataframe(dataframe = df[lines:], 
                directory    = image_dir,
                x_col        = "image_id",
                y_col        = "turtle_id",
                target_size  = (HEIGHT, WIDTH),
                batch_size   = BATCH_SIZE,
                classes      = CLASS_NAMES,
                class_mode   = 'categorical',
                shuffle      = True)
        
        return train_generator, validation_generator
    
    else:
        # Scale images for testing, no target provided and returned
        test_generator = test_datagen.flow_from_dataframe(dataframe = df, 
                directory   = image_dir,
                x_col       = "image_id",
                target_size = (HEIGHT, WIDTH),
                batch_size  = 1,
                class_mode  = None,
                shuffle     = False)
            
        return test_generator

## One model per image location (Transfer Learning using InceptionV3)

In [ ]:
# Clear any logs from previous runs
!rm -rf ./logs/

In [ ]:
#InceptionV3 layers
base_model = InceptionV3(input_shape = (HEIGHT, WIDTH, CHANNELS), include_top = False, weights = 'imagenet')

#Freeze layers for training
for layer in base_model.layers:
    layer.trainable = False

#Set Epochs for training
EPOCHS = 30

In [ ]:
#Model for images taken from LEFT side

#new layers for left image location
left = layers.Flatten()(base_model.output)
left = layers.Dense(1024, activation='relu')(left)
left = layers.Dropout(0.2)(left)

# Add a final softmax layer with 101 nodes for classification output
left = layers.Dense(NCLASSES, activation='softmax')(left)

model_left = tf.keras.models.Model(base_model.input, left)

#Compile model and set log and callbacks
model_left.compile(optimizer = tf.keras.optimizers.Adam(1e-5), loss = 'binary_crossentropy', 
                   metrics = tf.keras.metrics.TopKCategoricalAccuracy(k=5))
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#Load and preprocess data
train_datagen, test_datagen = preprocess()
train_generator_left, validation_generator_left = use_image_generator(train_data_left, 
                                                  train_datagen, test_datagen, training=True)

#Fit model
inception_left =  model_left.fit(
        train_generator_left, 
        validation_data=validation_generator_left,
        steps_per_epoch=TRAINING_SIZE // BATCH_SIZE, 
        epochs=EPOCHS,
        callbacks=[tensorboard_callback])

In [ ]:
#Model for images taken from RIGHT side

#new layers for right image location
right = layers.Flatten()(base_model.output)
right = layers.Dense(1024, activation='relu')(right)
right = layers.Dropout(0.2)(right)

# Add a final softmax layer with 101 nodes for classification output
right = layers.Dense(NCLASSES, activation='softmax')(right)

model_right = tf.keras.models.Model(base_model.input, right)

#Compile model and set log and callbacks
model_right.compile(optimizer = tf.keras.optimizers.Adam(1e-5), loss = 'binary_crossentropy', metrics = tf.keras.metrics.TopKCategoricalAccuracy(k=5))
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#Load and preprocess data
train_datagen, test_datagen = preprocess()
train_generator_right, validation_generator_right = use_image_generator(train_data_right, 
                                                    train_datagen, test_datagen, training=True)
    
#Fit model
inception_right =  model_right.fit(
        train_generator_right, 
        validation_data=validation_generator_right,
        steps_per_epoch=TRAINING_SIZE // BATCH_SIZE, 
        epochs=EPOCHS,
        callbacks=[tensorboard_callback])

In [ ]:
#Model for images taken from TOP side

#new layers for top image location
top = layers.Flatten()(base_model.output)
top = layers.Dense(1024, activation='relu')(top)
top = layers.Dropout(0.2)(top)

# Add a final softmax layer with 101 nodes for classification output
top = layers.Dense(NCLASSES, activation='softmax')(top)

model_top = tf.keras.models.Model(base_model.input, top)

#Compile model and set log and callbacks
model_top.compile(optimizer = tf.keras.optimizers.Adam(1e-5), loss = 'binary_crossentropy', metrics = tf.keras.metrics.TopKCategoricalAccuracy(k=5))
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#Load and preprocess data
train_datagen, test_datagen = preprocess()
train_generator_top, validation_generator_top = use_image_generator(train_data_top, 
                                                train_datagen, test_datagen, training=True)

#Fit model
inception_top =  model_top.fit(
        train_generator_top, 
        validation_data=validation_generator_top,
        steps_per_epoch=TRAINING_SIZE // BATCH_SIZE, 
        epochs=EPOCHS,
        callbacks=[tensorboard_callback])

In [ ]:
#Save models
#model_left.save('model_left')
#model_right.save('model_right')
#model_top.save('model_top')

## Prepare data for submission

In [ ]:
#Prepare prediction for test data
test_generator = use_image_generator(test_data, train_datagen, test_datagen, training=False)

In [ ]:
#Predict using three models for each location
y_preds = []
for index in range(len(test_data)):
    if test_data['image_location'][index] == 'left': 
        y_pred = model_left.predict(test_generator[index])
        y_preds.append(y_pred.flatten())
    elif test_data['image_location'][index] == 'right':
        y_pred = model_right.predict(test_generator[index])
        y_preds.append(y_pred.flatten())
    else:
        y_pred = model_top.predict(test_generator[index])
        y_preds.append(y_pred.flatten())

In [ ]:
#Get indices from top 5 predictions
# Corrected: [:,:-6:-1] instead of [:,-5:] so that best prediction comes first
y_preds = np.argsort(y_preds, axis=1)[:,:-6:-1]

#Save indices of top 5 predictions as dataframe
df = pd.DataFrame(y_preds)

In [ ]:
#Create a DataFrame with top 5 predictions in submission form
list = []
array = []
for line in y_preds:
    for id in line:
        list.append(CLASS_NAMES[id])
    array.append(list)
    list = []

titles = ['prediction1', 'prediction2','prediction3','prediction4','prediction5']

submission = pd.DataFrame(array, columns= titles)

#Insert image_ids from test_data
test_data = pd.read_csv('../data/test.csv') #image_id without appended .JPG
submission.insert(loc=0, column='image_id', value=test_data['image_id']) #Insert image_id in first column
submission

In [ ]:
#Save submission data as CSV
submission.to_csv('../data/submission.csv', index = False)